In [1]:
import pymysql
import pandas as pd
import numpy as np
import datetime
import time
import sys
sys.path.append(r'D:\jupyter files\waiting_time_project\my_tools')
import tools_for_os.for_df as ml_df

In [2]:
sql_re = """SELECT * FROM resource"""
data_re_ = ml_df.get_df_from_sql(sql_re)
print(f'The shape of data_resource {data_re_.shape}')
sql_re_appt = """SELECT * FROM resourceappointment"""
data_re_appt_ = ml_df.get_df_from_sql(sql_re_appt)
print(f'The shape of data_re_appt {data_re_appt_.shape}')
sql_appt = """SELECT * FROM appointment"""
data_appt_ = ml_df.get_df_from_sql(sql_appt)
print(f'The shape of data_appt {data_appt_.shape}')
sql_pl = """SELECT * FROM patientlocation"""
data_pl_ = ml_df.get_df_from_sql(sql_pl)
print(f'The shape of data_pl {data_pl_.shape}')
sql_plmh = """SELECT * FROM patientlocationmh"""
data_plmh_ = ml_df.get_df_from_sql(sql_plmh)
print(f'The shape of data_plmh {data_plmh_.shape}')

The shape of data_resource (477, 6)
The shape of data_re_appt (475920, 7)
The shape of data_appt (1074478, 17)
The shape of data_pl (206855, 9)
The shape of data_plmh (2333039, 10)


In [3]:
try:
    print('Drop columns')
    data_appt_.drop(columns = ['LastUpdated'], inplace = True)
    data_re_.drop(columns = ['LastUpdated'], inplace = True)
    data_re_appt_.drop('LastUpdated', axis = 1, inplace = True)
    data_pl_.drop(columns = ['LastUpdated'], inplace = True)
    data_plmh_.drop(columns = ['LastUpdated'], inplace = True)
except:
    print('Finish droppping columns')

Drop columns


In [4]:
data_re = data_re_.copy()
data_re_appt = data_re_appt_.copy()
data_appt = data_appt_.copy()
data_pl = data_pl_.copy()
data_plmh = data_plmh_.copy()

In [5]:
data_path = 'D:\\jupyter files\\data_waiting_time_project\\preprocess_data\\'

data_part1 = pd.read_csv(data_path + 'data_part1.csv', index_col = 0)
data_part2 = pd.read_csv(data_path + 'data_part2.csv', index_col = 0)

C:\Users\WENDY\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
# 把字符串转成datetime
def str_to_Datetime(st):
    dt = datetime.datetime.strptime(st, "%Y-%m-%d %H:%M:%S")
    return dt

已经检查过，一个Appointment 里面只有一个AliasSerNum

In [7]:
print(f'The shape of data_appt {data_appt.shape}')
data_appt = data_appt[
    (data_appt.ActualStartDate != datetime.datetime(1970, 1, 1, 0, 0)) &
    (data_appt.ActualEndDate != datetime.datetime(1970, 1, 1, 0, 0)) &
    (data_appt.ActualStartDate != '0000-00-00 00:00:00')]
print(f'The shape of data_appt {data_appt.shape}')

The shape of data_appt (1074478, 16)
The shape of data_appt (370299, 16)


一个AliasSerNum 对应多个ResourceSerNum
一个ResourceSerNum 对应多个AppointmentSerNum

In [8]:
print('Merge data_appt and data_re_appt')
appt_re_appt = pd.merge(data_appt, data_re_appt, on = ['AppointmentSerNum', 'AliasSerNum'], how = 'inner')
print(f'The shape of appt_re_appt {appt_re_appt.shape}')
appt_re_appt.drop('AliasSerNum', axis = 1, inplace = True)

print('\nMerge data_re')
appt_loc = pd.merge(data_re, appt_re_appt, on = ['ResourceSerNum'], how = 'inner')
print(f'The shape of appt_loc {appt_loc.shape}')

print('\nMerge data_pl')
data_pl.drop('AliasSerNum', axis = 1, inplace = True)
appt_loc_pl = pd.merge(data_pl, appt_loc, on = ['AppointmentSerNum'], how = 'right')
print(f'The shape of appt_loc_pl {appt_loc_pl.shape}')

print('\nGet Treat_Date')
appt_loc_pl['Treat_Date'] = appt_loc_pl.apply(lambda x: x.ActualStartDate.strftime("%Y-%m-%d"), axis = 1)

Merge data_appt and data_re_appt
The shape of appt_re_appt (165512, 20)

Merge data_re
The shape of appt_loc (165512, 23)

Merge data_pl
The shape of appt_loc_pl (165512, 29)

Get Treat_Date


In [9]:
appt_loc_pl.head()

,PatientLocationSerNum,AppointmentSerNum,PatientLocationAriaSer,ResourceSer,RevCount,CheckedInFlag,ArrivalDateTime,ResourceSerNum,ResourceAriaSer,ResourceName,...,ScheduledStartTime,ScheduledEndTime,ActualStartDate,ActualEndDate,EstimatedDuration,HstryUserName,ResourceAppointmentSerNum,ExclusiveFlag,PrimaryFlag,Treat_Date
0,30872.0,849926,410426.0,8332.0,9.0,1.0,2015-10-19 14:21:00,176,7740,TB_6,...,2015-10-19 14:00:00,2015-10-19 14:15:00,2015-10-19 14:18:00,2015-10-19 14:34:00,0.0,amendelsohn,41455,1,0,2015-10-19
1,30873.0,849927,410642.0,8330.0,9.0,1.0,2015-10-20 13:27:00,177,7738,TB_4,...,2015-10-20 13:45:00,2015-10-20 14:00:00,2015-10-20 13:32:00,2015-10-20 13:48:00,0.0,esirois,41456,1,0,2015-10-20
2,30874.0,849928,410894.0,8332.0,9.0,1.0,2015-10-21 14:43:00,176,7740,TB_6,...,2015-10-21 15:30:00,2015-10-21 15:45:00,2015-10-21 14:57:00,2015-10-21 15:07:00,0.0,hli,41457,1,0,2015-10-21
3,30875.0,849929,411106.0,8332.0,9.0,1.0,2015-10-22 13:35:00,176,7740,TB_6,...,2015-10-22 14:00:00,2015-10-22 14:15:00,2015-10-22 13:51:00,2015-10-22 13:59:00,0.0,amendelsohn,41458,1,0,2015-10-22
4,30876.0,849930,411281.0,8332.0,9.0,1.0,2015-10-23 12:36:00,176,7740,TB_6,...,2015-10-23 13:00:00,2015-10-23 13:15:00,2015-10-23 12:52:00,2015-10-23 13:01:00,0.0,amendelsohn,41459,1,0,2015-10-23


In [7]:
appt_loc_pl_grouped = appt_loc_pl.groupby('ResourceSerNum')
appt_loc_pl_grouped.groups.keys()

dict_keys([145, 175, 176, 177, 178, 179, 180, 237, 281, 293, 301, 303, 304, 310, 313, 359, 363, 365, 366, 369, 370, 371, 372, 373, 391, 406, 409, 416, 437, 666, 672, 697])

In [8]:
sample = appt_loc_pl_grouped.get_group(237)
sample.sort_values(by = 'ActualStartDate', inplace = True)
print(sample.shape)

(2930, 30)


C:\Users\WENDY\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [9]:
sample.columns

Index(['PatientLocationSerNum', 'AppointmentSerNum', 'PatientLocationAriaSer',
       'ResourceSer', 'RevCount', 'CheckedInFlag', 'ArrivalDateTime',
       'ResourceSerNum', 'ResourceAriaSer', 'ResourceName', 'AliasSerNum',
       'ResourceType', 'PatientSerNum', 'AppointmentAriaSer',
       'ActivityInstanceAriaSer', 'DiagnosisSerNum', 'PrioritySerNum',
       'AliasExpressionSerNum', 'Status', 'State', 'ScheduledStartTime',
       'ScheduledEndTime', 'ActualStartDate', 'ActualEndDate',
       'EstimatedDuration', 'HstryUserName', 'ResourceAppointmentSerNum',
       'ExclusiveFlag', 'PrimaryFlag', 'Treat_Date'],
      dtype='object')

In [10]:
sample_grouped = sample.groupby('Treat_Date')
sample_grouped.groups.keys()

dict_keys(['2014-01-30', '2015-01-05', '2015-01-06', '2015-01-07', '2015-01-08', '2015-01-09', '2015-01-12', '2015-01-13', '2015-01-14', '2015-01-15', '2015-01-16', '2015-01-18', '2015-01-19', '2015-01-20', '2015-01-21', '2015-01-22', '2015-01-23', '2015-01-26', '2015-01-27', '2015-01-28', '2015-01-29', '2015-01-30', '2015-02-02', '2015-02-03', '2015-02-04', '2015-02-05', '2015-02-06', '2015-02-09', '2015-02-10', '2015-02-11', '2015-02-12', '2015-02-13', '2015-02-17', '2015-02-18', '2015-02-19', '2015-02-20', '2015-02-23', '2015-02-24', '2015-02-25', '2015-02-26', '2015-02-27', '2015-03-02', '2015-03-03', '2015-03-04', '2015-03-05', '2015-03-06', '2015-03-09', '2015-03-10', '2015-03-11', '2015-03-12', '2015-03-13', '2015-03-16', '2015-03-17', '2015-03-18', '2015-03-19', '2015-03-20', '2015-03-23', '2015-03-24', '2015-03-25', '2015-03-26', '2015-03-27', '2015-03-29', '2015-03-30', '2015-03-31', '2015-04-01', '2015-04-02', '2015-04-07', '2015-04-08', '2015-04-09', '2015-04-10', '2015-04-

In [39]:
room = sample_grouped.get_group('2015-05-29')
room = room[[
    'AppointmentSerNum', 'PatientSerNum', 'AliasExpressionSerNum',
    'ScheduledStartTime', 'ScheduledEndTime',
    'ActualStartDate', 'ActualEndDate',]]
room.sort_values(by = 'ActualStartDate', inplace = True)

## 当前一天的数据

In [67]:
data_part1['ScheduledStartTime'] = data_part1.ScheduledStartTime.apply(lambda x: str_to_Datetime(x))
print(f'The shape of data_part1 is {data_part1.shape}')
print(f'The shape of data_part2 is {data_part2.shape}')

DATA = pd.merge(data_part1, data_part2, on = ['PatientSerNum', 'date'], how = 'inner')
DATA.sort_values(by = ['PatientSerNum', 'AppointmentSerNum', 'ScheduledStartTime', 'FractionNumber'], inplace = True)
print(f'The shape of data is {DATA.shape}')

The shape of data_part1 is (1047176, 21)
The shape of data_part2 is (507863, 16)
The shape of data is (1788434, 35)


In [68]:
DATA.head()

,dxt_AliasName,DateOfBirth,Sex,AliasSerNum,PatientSerNum,AppointmentSerNum,ScheduledStartTime,ScheduledEndTime,ActualStartDate,ActualEndDate,...,ImagesTaken,UserName,RadiationSerNum,RadiationId,ResourceSerNum,MU,MUCoeff,TreatmentTime,CourseId,Treatment_duration
0,METS,1935-10-27 00:00:00,Male,31,83,1490213,2018-06-14 16:45:00,2018-06-14 17:30:00,2018-06-14 14:33:00,2018-06-14 14:55:00,...,0,rislam,632257,1.1 ANT,177,432.523,109.293,0.002833,C1,0
1,METS,1935-10-27 00:00:00,Male,31,83,1490213,2018-06-14 16:45:00,2018-06-14 17:30:00,2018-06-14 14:33:00,2018-06-14 14:55:00,...,0,rislam,632257,1.1 ANT,177,432.523,109.293,0.002833,C1,0
2,METS,1935-10-27 00:00:00,Male,31,83,1490213,2018-06-14 16:45:00,2018-06-14 17:30:00,2018-06-14 14:33:00,2018-06-14 14:55:00,...,0,rislam,632258,1.2 POST,177,434.852,109.881,0.002833,C1,0
3,METS,1935-10-27 00:00:00,Male,31,83,1490213,2018-06-14 16:45:00,2018-06-14 17:30:00,2018-06-14 14:33:00,2018-06-14 14:55:00,...,0,rislam,632258,1.2 POST,177,434.852,109.881,0.724500,C1,60
4,METS,1935-10-27 00:00:00,Male,31,83,1490213,2018-06-14 16:45:00,2018-06-14 17:30:00,2018-06-14 14:33:00,2018-06-14 14:55:00,...,0,rislam,632257,1.1 ANT,177,432.523,109.293,0.720617,C1,0


In [53]:
room['pre_ActualEndDate'] = room.ActualEndDate.shift(periods = 1, fill_value = room.ActualStartDate	.iloc[0])
room['Transfer_duration'] = room.apply(lambda x: (x.ActualStartDate - x.pre_ActualEndDate).seconds, axis = 1)
room['Interval_number'] = room.apply(lambda x: int(x.Transfer_duration / 60 / 15), axis = 1)
room

,AppointmentSerNum,PatientSerNum,AliasExpressionSerNum,ScheduledStartTime,ScheduledEndTime,ActualStartDate,ActualEndDate,pre_ActualEndDate,Transfer_duration,Interval_number
154702,748067,30108,1522,2015-05-29 07:45:00,2015-05-29 08:15:00,2015-05-29 07:40:00,2015-05-29 08:30:00,2015-05-29 07:40:00,0,0
157345,844081,38166,1505,2015-05-29 09:00:00,2015-05-29 09:15:00,2015-05-29 08:31:00,2015-05-29 09:07:00,2015-05-29 08:30:00,60,0
155924,835994,37310,1505,2015-05-29 09:30:00,2015-05-29 09:45:00,2015-05-29 09:07:00,2015-05-29 09:22:00,2015-05-29 09:07:00,0,0
157450,846312,38504,1505,2015-05-29 09:45:00,2015-05-29 10:00:00,2015-05-29 09:28:00,2015-05-29 09:53:00,2015-05-29 09:22:00,360,0
155198,827677,36509,1505,2015-05-29 10:30:00,2015-05-29 10:45:00,2015-05-29 09:53:00,2015-05-29 10:17:00,2015-05-29 09:53:00,0,0
157295,843775,38104,1505,2015-05-29 09:15:00,2015-05-29 09:30:00,2015-05-29 10:30:00,2015-05-29 11:00:00,2015-05-29 10:17:00,780,0
157300,843872,38137,1505,2015-05-29 10:45:00,2015-05-29 11:00:00,2015-05-29 11:01:00,2015-05-29 11:15:00,2015-05-29 11:00:00,60,0
156772,839759,37654,1505,2015-05-29 11:00:00,2015-05-29 11:15:00,2015-05-29 11:16:00,2015-05-29 11:32:00,2015-05-29 11:15:00,60,0
156789,839883,37675,1505,2015-05-29 13:45:00,2015-05-29 14:00:00,2015-05-29 13:41:00,2015-05-29 14:00:00,2015-05-29 11:32:00,7740,8
157079,841751,37864,1505,2015-05-29 23:30:00,2015-05-29 23:45:00,2015-05-29 14:38:00,2015-05-29 15:01:00,2015-05-29 14:00:00,2280,2


In [70]:
type(room.ScheduledStartTime.iloc[0])

datetime.datetime

In [ ]:
sample_grouped.get_group('2015-05-29')[[
    'AppointmentSerNum', 'PatientSerNum', 'AppointmentAriaSer',
    'ActivityInstanceAriaSer', 'DiagnosisSerNum', 'PrioritySerNum',
    'AliasExpressionSerNum', 'ScheduledStartTime', 'ScheduledEndTime',
    'ActualStartDate', 'ActualEndDate', 'ResourceAppointmentSerNum']]